In [5]:
sql ='SELECT T1.tvmaze_person_id FROM tvmaze_casting AS T1 WHERE T1.tvmaze_person_id != (SELECT T22.tvmaze_id FROM tvmaze_casting AS T22  GROUP BY T22.tvmaze_id  ORDER BY count( T22.tvmaze_character_id) DESC LIMIT 1)'

from sql2question_parallel import sql2question

result  = sql2question(sql, 'tvmaze')
result

KeyboardInterrupt: 

In [ ]:
from sql2question_parallel import en2ch
en = "Who are the cast members whose ID does not match the ID of the person with the most characters played in a TV show?"
result2 = en2ch(en)
result2

'哪些演员的ID与在电视节目中扮演角色最多的人的ID不匹配？'

In [ ]:
import json 

with open('token_count.json', 'r') as f:
    token_count = json.load(f)

inputcount = 0
outputcount = 0
for item in token_count['sql2en']:
    inputcount1 = sum(item['input'])
    outputcount1 = sum(item['output'])
    inputcount += inputcount1
    outputcount += outputcount1
print(inputcount, outputcount)

469595 30865


In [ ]:
inputcount = 0
outputcount = 0
for item in token_count['en2ch']:
    inputcount1 = item['input']
    outputcount1 = item['output']
    inputcount += inputcount1
    outputcount += outputcount1
print(inputcount, outputcount)

92601 5800


In [ ]:
def count_tokens(input_string):
    # 使用空格将字符串拆分为单词列表
    tokens = input_string.split()
    # 返回单词列表的长度
    return len(tokens)

# 测试字符串
input_string = "这是一个示例字符串，包含一些token。"
# 统计token数
token_count = count_tokens(input_string)
print("Token数:", token_count)


In [3]:
from sql2question_group import preprocess, get_clause, after_process, rough_tranlate
from src.qa import makecomversation
import json
with open("./data/4_26/complex/Accidents_complex.json", "r") as f:
    data = json.load(f)

# 粗翻译,翻译到接近自然语言,没有特殊标记符号,和类似函数的表达形式.  200 tokens
def rough_tranlate_clause(sentences):
    prompt = """You are a Translator."""

    usercontent = f""" 
    Translate the sentences provided into spoken English, not contain any special symbols except commas and periods.
    Only returns the translated sentences, do not generate any other content, such as "The caluse is: ...".
    if you have trouble doing it or hold that nothing need to be changed, just return the original sentences,
    the returned sentences is either the Translated sentences or the original sentences, but we encourage you to translate the sentences even if the changes are minor.
    The sentences is: 
    {sentences}
    """

    return_format = '{"sentence1":"","sentence2":"", ...}'

    result, input_tokens, ouput_tokens = makecomversation(prompt, usercontent, return_format)
    
    return result


def rough_tranlate(clause_lists):
    clauses = {}
    i = 1
    for clause_list in clause_lists:
        for clause in clause_list:
            clauses[f'clauses{i}'] = clause
            i+=1
  
    result = rough_tranlate_clause(clauses)
    print(result)
    i = 1
    result2 = []
    for clause_list in clause_lists:
        result1 = []
        for clause in clause_list:
            result1.append(result[f'sentence{i}'])
            i+=1
        result2.append(result1)
    return result2

# 预处理
sqls = []
for index, item in enumerate(data[:10]):
    sqls.append(item['readable_query'])
preprocessed_sqls = []
for sql in sqls:
    sql, _, _, _, _ = preprocess(sql)
    preprocessed_sqls.append(sql)
clause_lists = []
for sql in preprocessed_sqls:
    clause_list = get_clause(sql)
    clause_lists.append(clause_list)
afterprocessed_clauses = []
for clause_list in clause_lists:
    decomposed_clause = after_process(clause_list)
    afterprocessed_clauses.append(decomposed_clause)
simplified_sql = rough_tranlate(afterprocessed_clauses)

{'sentence1': 'Select the breathalyzer test result from the person table.', 'sentence2': "Where the person's accident ID is not in the accident table.", 'sentence3': 'Select the longitude from the accident table.', 'sentence4': 'Where the accident ID is not in the person table.', 'sentence5': 'Select the count of breathalyzer tests from the person table.', 'sentence6': 'Where the administrative unit of the person is not in the accident table.', 'sentence7': 'Select the average breathalyzer test result and the description of the scene from the accident table.', 'sentence8': 'Group by the description of the scene.', 'sentence9': 'Select the average professional examination and the name of the administrative unit from the administrative unit table.', 'sentence10': 'Group by the name of the administrative unit.', 'sentence11': 'Select the average latitude and the name of the administrative unit from the administrative unit table.', 'sentence12': 'Group by the name of the administrative uni

In [11]:
from sql2question_group import get_examples

def fewshot_tranlate_clause(clauses, clause_types):
    if len(clause_types) > 2:
        sameple_num = 5
    else:
        sameple_num = 10
    examples = []
    for clause_type in clause_types:
        examples += get_examples(clause_type, sameple_num)

    example_prompt = "Here is some examples:" if len(examples)>0 else ""
    examples = "" if len(examples) == 0 else examples
    prompt = f"""You are a language expert. You need to translate the sql clauses provided and then return the colloquial result. 
    {example_prompt}
    {examples}
    """

    usercontent = f""" You need to translate the sql clauses provided and then return the colloquial result. 
    Do not generate any other content, such as "The paragraph means: ", just the result. 
    if you can't translate the sql clauses, return the clauses provided, don't reply content as such"'The SQL clauses is not valid. Please provide a valid SQL clauses.'".
    The sql clauses is: 
    {clauses}
    """

    return_format = '{"sentence1":"","sentence2":"", ...}'

    result, input_tokens, ouput_tokens = makecomversation(prompt, usercontent, return_format)
    return result


def get_hint(clause_lists):
    clauses = {}
    clause_types = []
    i = 1
    for clause_list in clause_lists:
        for clause in clause_list:
            clauses[f'clauses{i}'] = clause
            clause_type = clause.split()[0].lower()
            clause_types.append(clause_type)
            i+=1
    
    result = fewshot_tranlate_clause(clauses, list(set(clause_types)))
    print(result)
    i = 1
    result2 = []
    for clause_list in clause_lists:
        result1 = []
        for clause in clause_list:
            result1.append(result[f'sentence{i}'])
            i+=1
        result2.append(result1)
    return result2

hints = get_hint(simplified_sql)

hints


{'sentence1': 'what are the breathalyzer test results', 'sentence2': 'for people whose accident ID is not listed in the accident table', 'sentence3': 'what is the longitude', 'sentence4': 'for accidents not listed in the person table', 'sentence5': 'how many breathalyzer tests are there', 'sentence6': 'for people whose administrative unit is not listed in the accident table', 'sentence7': 'what is the average breathalyzer test result and the scene description', 'sentence8': 'for each scene description', 'sentence9': 'what is the average professional examination and the administrative unit name', 'sentence10': 'for each administrative unit', 'sentence11': 'what is the average latitude and the administrative unit name', 'sentence12': 'for each administrative unit', 'sentence13': 'what is the average latitude and the population of the administrative unit', 'sentence14': 'for each population size of the administrative unit', 'sentence15': 'what is the average longitude and the population o

[['what are the breathalyzer test results',
  'for people whose accident ID is not listed in the accident table'],
 ['what is the longitude', 'for accidents not listed in the person table'],
 ['how many breathalyzer tests are there',
  'for people whose administrative unit is not listed in the accident table'],
 ['what is the average breathalyzer test result and the scene description',
  'for each scene description'],
 ['what is the average professional examination and the administrative unit name',
  'for each administrative unit'],
 ['what is the average latitude and the administrative unit name',
  'for each administrative unit'],
 ['what is the average latitude and the population of the administrative unit',
  'for each population size of the administrative unit'],
 ['what is the average longitude and the population of the administrative unit',
  'for each population size of the administrative unit'],
 ['what is the average latitude and the age of the person',
  'for each age group

In [23]:
import random


def the_final_translate(original_sqls, simplified_versions, hints, descriptions):
    with open('spider_examples.txt', 'r', encoding='utf-8') as f:
        spider_examples = f.readlines()
    random_selection = random.sample(spider_examples, 10)

    is_descriptions = descriptions is not None and len(descriptions) > 0
    
    prompt = f"""You are a sqls to question translation expert. You need to translate the sqls provided and then return the coresponding questions easy to understand. 
    Especially, when "count(stuff_id)" or "group by stuff_id", we means "how many stuff are there", or "for each stuff", not how many ids are there, don't mention id or IDS if not necessary. 
    and don't use the word "count" or orther function name in the question, say what they do.
    Also, don't mention the table name or column name in the question, or use the word "table" or "column" in the question, guess the realistic meaning from the table name, 
    such as table "address.address" is just a address, the "address.last_update" is just the last update time.
    don't use from clause in the question, and don't use the word "from" in the question, people who ask the question don't need to know where the data is stored.
    
    here is some examples:
    {random_selection}
    """

    usercontent = f""" You need to translate the sqls provided and then return coresponding questions. 
    Because sometimes sql is too complex to understand, or even contains errors, we will also provide a simplified version(a clause list, contains main information about the sql) for reference. 
    In addition, we will also provide some tips to introduce the meaning of this sqls, and what the generated question is mostly about.
    Do not generate any other content, such as "The question is: ", just the question of the sqls itself. 
    if you can't translate the sqls, return the simplified version provided(needs to be concat in right order), don't reply content as such "The sqls clause is not valid. Please provide a valid SQL clause.".
    """ 

    if is_descriptions:
        temp = f"""
        We will provide the descriptions of the columns in sqls, which you can take as reference.
        """
        usercontent += temp
    
    inputs = {}
    i = 1
    for original_sql, simplified_version, hint in zip(original_sqls, simplified_versions, hints):
        inputs[f'batch{i}'] = {}
        inputs[f'batch{i}']['original_sql'] = original_sql
        inputs[f'batch{i}']['simplified_sql'] = simplified_version
        inputs[f'batch{i}']['question_hint'] = hint
        i += 1
    print(inputs)
    usercontent += f"""
    The sqls are: 
    {inputs}
    """

    if is_descriptions:
        usercontent += f"""
        The descriptions of the columns are:
        {descriptions}
        """

    return_format = '{"question1":"","question2":"", ...}'

    result, input_tokens, ouput_tokens = makecomversation(prompt, usercontent, return_format)
    print(result)
    result1 = []
    i =1
    for item in result:
        result1.append(result[f'question{i}'])
        i+=1
    return result1

descriptions = 'no descriptions available, you have to guess from the table name.'
simplified_sql
hints
questions = the_final_translate(sqls, simplified_sql, hints, descriptions)

{'batch1': {'original_sql': 'SELECT oseba.alkotest FROM oseba AS oseba WHERE oseba.id_nesreca NOT IN (SELECT nesreca.id_nesreca FROM nesreca AS nesreca)', 'simplified_sql': ['Select the breathalyzer test result from the person table.', "Where the person's accident ID is not in the accident table."], 'question_hint': ['what are the breathalyzer test results', 'for people whose accident ID is not listed in the accident table']}, 'batch2': {'original_sql': 'SELECT nesreca.x_wgs84 FROM nesreca AS nesreca WHERE nesreca.id_nesreca NOT IN (SELECT oseba.id_nesreca FROM oseba AS oseba)', 'simplified_sql': ['Select the longitude from the accident table.', 'Where the accident ID is not in the person table.'], 'question_hint': ['what is the longitude', 'for accidents not listed in the person table']}, 'batch3': {'original_sql': 'SELECT count( oseba.alkotest_d) FROM oseba AS oseba WHERE oseba.upravna_enota NOT IN (SELECT nesreca.upravna_enota FROM nesreca AS nesreca)', 'simplified_sql': ['Select th

In [32]:
questions # 还算ok

['What are the breathalyzer test results for people whose accident ID is not listed in the accident table?',
 'What is the longitude for accidents not listed in the person table?',
 'How many breathalyzer tests are there for people whose administrative unit is not listed in the accident table?',
 'What is the average breathalyzer test result and the scene description for each scene description?',
 'What is the average professional examination and the administrative unit name for each administrative unit?',
 'What is the average latitude and the administrative unit name for each administrative unit?',
 'What is the average latitude and the population of the administrative unit for each population size of the administrative unit?',
 'What is the average longitude and the population of the administrative unit for each population size of the administrative unit?',
 'What is the average latitude and the age of the person for each age group?',
 'What is the average longitude and the gender o

In [30]:
# random fewshot 翻译 300tokens
def getAnswer(ens):
    inputs = {}
    i = 1
    for en in ens:
        inputs[f'english{i}'] = en
        i+=1

    with open('./CSpider/en2ch_examples.json', 'r', encoding='utf-8') as f:
        spider_examples = json.load(f)
    random_selection = random.sample(spider_examples, 10)

    examples = ''
    index = 1
    for item in random_selection:
        example = f"examples{index}: \"{item['english question']}\", its Chinese translation is \"{item['chinese question']}\".\n"
        examples += example
        index += 1

    prompt = f"""You are a translation expert. You need to translated the english question to chinese question. 
    当你看到单词unique例如'unique ticket numbers'，它通常是"不同的"意思， 例如
    "for each unique aircraft code" 可以翻译为"不同飞机代码" 而不是"每个独特的飞机代码"。
    here are some examples:
    {examples}
    """

    usercontent = f""" Translate the chinese questions provided below, then return the translated english questions.
    The english questions is: {inputs} 
    """

    return_format = '{"chinese1":"","chinese2":"", ...}'

    result, input_tokens, ouput_tokens = makecomversation(prompt, usercontent, return_format)
    
    result1 = []
    for i, item in enumerate(result):
        result1.append(result[f'chinese{i+1}'])
    return result1

chs = getAnswer(questions)